# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [2]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

Cloning into 'GroceryStoreDataset'...
remote: Enumerating objects: 6559, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 6559 (delta 45), reused 35 (delta 35), pack-reused 6293
Receiving objects: 100% (6559/6559), 116.26 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (275/275), done.


In [4]:
from pathlib import Path
from PIL import Image
from typing import List, Tuple

import torch
from torch import Tensor,device
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
import torchvision

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

import random
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

Device: cuda


In [5]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [6]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

In [7]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_dataset = GroceryStoreDataset(split='train',transform=transform)
val_dataset = GroceryStoreDataset(split='val',transform=transform)
test_dataset = GroceryStoreDataset(split='test',transform=transform)
train_dataset.__len__(),val_dataset.__len__(),test_dataset.__len__()

(2640, 296, 2485)

In [9]:
def check_image_shapes(dataset):
    for i in range(dataset.__len__()):
        img, label = dataset[i]
        w, h = 0,0
        #print(f"Image {i} shape: {img.shape} (width, height)")
        if img.shape[1] > w:
          w = img.shape[1]
        if img.shape[2] > h:
          h = img.shape[2]
    return w,h
# Check shapes of the first few images
w_t,h_t = check_image_shapes(train_dataset)
w_t,h_t
#w_v,h_v = check_image_shapes(val_dataset)
#w_T,h_T = check_image_shapes(test_dataset)

(464, 348)

In [10]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((348, 348)),
    torchvision.transforms.ToTensor(),
])
train_dataset = GroceryStoreDataset(split='train',transform=transform)
batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [11]:
mean = torch.zeros(3)
std = torch.zeros(3)

for images,_ in train_loader:
    mean += torch.mean(images, dim=(0, 2, 3))
    std += torch.std(images, dim=(0, 2, 3))
mean /= train_dataset.__len__()
std /= train_dataset.__len__()

print(f'Mean: {mean}')
print(f'Std: {std}')

Mean: tensor([0.5306, 0.3964, 0.2564])
Std: tensor([0.2325, 0.2093, 0.1781])


## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

In [12]:
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, val_dl=None, verbose=False, label_smoothing=0):
    best_val_acc = 0
    best_params = []
    best_epoch = -1

    for e in tqdm(range(epochs)):
        model.train()
        # Train
        train_loss = 0
        train_samples = 0
        train_acc = 0
        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)
            scores = model(imgs)
            loss = F.cross_entropy(scores, labels,label_smoothing=label_smoothing)
            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()

            opt.zero_grad()  # clear
            loss.backward()  # fill
            opt.step()       # use

        train_acc /= train_samples
        train_loss /= train_samples

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            val_acc = 0
            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)
                    val_scores = model(imgs)
                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]
                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()
                val_acc /= val_samples
                val_loss /= val_samples

            if val_dl is None or val_acc > best_val_acc:
                best_val_acc = val_acc if val_dl is not None else 0
                best_params = model.state_dict()
                torch.save(best_params, "best_model.pth")
                best_epoch = e

        if verbose: #and e % 5 == 0:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return best_val_acc, best_params, best_epoch

# CNN

In [13]:
image_height, image_width, image_channels = 224,224,3 #348, 348, 3
num_classes = train_dataset.get_num_classes()

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((348, 348)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    #Mean: tensor([0.5306, 0.3964, 0.2564])    #Std: tensor([0.2325, 0.2093, 0.1781])
    torchvision.transforms.Normalize((0.5, 0.4, 0.25), (0.23, 0.2, 0.17))
])

transform_2 = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size=(224, 224),antialias=True),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.4, 0.25), (0.23, 0.2, 0.17))
])
train_dataset = GroceryStoreDataset(split='train',transform=transform_2)
val_dataset = GroceryStoreDataset(split='val',transform=transform_2)
test_dataset = GroceryStoreDataset(split='test',transform=transform_2)

batch = 8
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True)

In [14]:
class ExtendedCNN(nn.Module):
    def __init__(self,channels=16, num_classes=num_classes):
        super(ExtendedCNN, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels*2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=channels*2, out_channels=channels*4, kernel_size=3, stride=1, padding=1)

        # Calculate the size of the feature map after the convolutional layers
        def conv2d_size_out(size, kernel_size=3, stride=1, padding=1):
            return (size - kernel_size + 2 * padding) // stride + 1
        def conv2d_compute_size(conv_number,kernels,max_poolings):  #max pooling ex. [1,0,1] if 1 them there is a max_pool
          image_w = image_width
          for i in range(conv_number):
            image_w = conv2d_size_out(image_w, kernel_size=kernels[i])
            if max_poolings[i] == 1:
              image_w = image_w//2
          return image_w

        #convh = conv2d_size_out(conv2d_size_out(image_height, kernel_size=5, padding=1) // 2, kernel_size=3, padding=1) // 2
        image_w = conv2d_compute_size(3,[3,3,3],[1,1,1])

        linear_input_size = image_w * image_w * channels*4  # Adjusted based on the output channels of conv2

        # Define fully connected layers
        self.fc1 = nn.Linear(linear_input_size, 128)
        #self.drop1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        #self.drop2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        #x = self.drop1(x)
        x = F.relu(self.fc2(x))
        #x = self.drop2(x)
        x = self.fc3(x)

        return x


In [15]:
new_model = ExtendedCNN(
    num_classes = num_classes,
)
new_model.to(device)

optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001,weight_decay=0.01)
epochs = 50
label_smoothing = 0.01
best_val_acc, best_params, best_epoch = train_loop(
    new_model,
    train_loader,
    epochs,
    optimizer,
    val_loader,
    verbose=True,
    label_smoothing = label_smoothing,
)

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Gridsearch

In [46]:
weight_decays = [0.0, 0.01, 0.001]
label_smoothings = [0.01, 0.1,0.5]
lrs = [0.001, 0.0001]
channels = 16
batch = 8

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)
epochs = 40
# Perform grid search
for weight_decay in weight_decays:
    for label_smoothing in label_smoothings:
        for lr in lrs:
            model = ExtendedCNN(channels=channels, num_classes=num_classes)
            model.to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

            print(f"Training with: weight_decay={weight_decay}, label_smoothing={label_smoothing}, lr={lr}, channels={channels}, batch_size={batch_size}")
            train_loop(
                model,
                train_loader,
                epochs,
                optimizer,
                val_loader,
                verbose=True,
                label_smoothing = label_smoothing,
            )

Training with: weight_decay=0.0, label_smoothing=0.0, lr=0.001, channels=16, batch_size=8


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 0: train loss 2.911 - train acc 0.202 - valid loss 2.900 - valid acc 0.226
Epoch 1: train loss 2.225 - train acc 0.328 - valid loss 2.608 - valid acc 0.311
Epoch 2: train loss 1.857 - train acc 0.416 - valid loss 2.811 - valid acc 0.301
Epoch 3: train loss 1.698 - train acc 0.443 - valid loss 2.360 - valid acc 0.270
Epoch 4: train loss 1.553 - train acc 0.485 - valid loss 2.339 - valid acc 0.331
Epoch 5: train loss 1.420 - train acc 0.525 - valid loss 2.383 - valid acc 0.284
Epoch 6: train loss 1.337 - train acc 0.564 - valid loss 2.187 - valid acc 0.382
Epoch 7: train loss 1.285 - train acc 0.557 - valid loss 2.082 - valid acc 0.358
Epoch 8: train loss 1.181 - train acc 0.608 - valid loss 2.430 - valid acc 0.355
Epoch 9: train loss 1.209 - train acc 0.600 - valid loss 2.016 - valid acc 0.405
Epoch 10: train loss 1.073 - train acc 0.638 - valid loss 2.107 - valid acc 0.372
Epoch 11: train loss 1.071 - train acc 0.638 - valid loss 2.266 - valid acc 0.389
Epoch 12: train loss 0.967

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 0: train loss 3.264 - train acc 0.147 - valid loss 3.187 - valid acc 0.162
Epoch 1: train loss 2.986 - train acc 0.181 - valid loss 3.204 - valid acc 0.169
Epoch 2: train loss 2.919 - train acc 0.202 - valid loss 3.120 - valid acc 0.193
Epoch 3: train loss 2.883 - train acc 0.208 - valid loss 3.096 - valid acc 0.159
Epoch 4: train loss 3.189 - train acc 0.170 - valid loss 3.527 - valid acc 0.074
Epoch 5: train loss 3.435 - train acc 0.105 - valid loss 3.490 - valid acc 0.074
Epoch 6: train loss 3.413 - train acc 0.102 - valid loss 3.503 - valid acc 0.074
Epoch 7: train loss 3.412 - train acc 0.105 - valid loss 3.502 - valid acc 0.074
Epoch 8: train loss 3.412 - train acc 0.105 - valid loss 3.504 - valid acc 0.074
Epoch 9: train loss 3.413 - train acc 0.103 - valid loss 3.506 - valid acc 0.074
Epoch 10: train loss 3.411 - train acc 0.105 - valid loss 3.504 - valid acc 0.074
Epoch 11: train loss 3.412 - train acc 0.105 - valid loss 3.500 - valid acc 0.074
Epoch 12: train loss 3.412

...

Epoch 0: train loss 3.696 - train acc 0.124 - valid loss 3.309 - valid acc 0.095
Epoch 1: train loss 3.123 - train acc 0.159 - valid loss 3.248 - valid acc 0.152
Epoch 2: train loss 2.995 - train acc 0.172 - valid loss 3.181 - valid acc 0.118
Epoch 3: train loss 2.962 - train acc 0.185 - valid loss 3.197 - valid acc 0.176
Epoch 4: train loss 2.956 - train acc 0.175 - valid loss 3.143 - valid acc 0.125
Epoch 5: train loss 2.936 - train acc 0.185 - valid loss 3.226 - valid acc 0.145
Epoch 6: train loss 2.930 - train acc 0.195 - valid loss 3.355 - valid acc 0.098
Epoch 7: train loss 2.931 - train acc 0.195 - valid loss 3.216 - valid acc 0.095
Epoch 8: train loss 2.924 - train acc 0.198 - valid loss 3.179 - valid acc 0.145
Epoch 9: train loss 4.750 - train acc 0.165 - valid loss 3.529 - valid acc 0.098
Epoch 10: train loss 3.447 - train acc 0.102 - valid loss 3.488 - valid acc 0.074
Epoch 11: train loss 3.445 - train acc 0.105 - valid loss 3.491 - valid acc 0.074
Epoch 12: train loss 3.419

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 0: train loss 3.130 - train acc 0.203 - valid loss 2.834 - valid acc 0.274
Epoch 1: train loss 2.599 - train acc 0.333 - valid loss 2.499 - valid acc 0.301
Epoch 2: train loss 2.366 - train acc 0.392 - valid loss 2.462 - valid acc 0.280
Epoch 3: train loss 2.211 - train acc 0.445 - valid loss 2.338 - valid acc 0.331
Epoch 4: train loss 2.126 - train acc 0.461 - valid loss 2.352 - valid acc 0.277
Epoch 5: train loss 2.021 - train acc 0.504 - valid loss 2.088 - valid acc 0.392
Epoch 6: train loss 1.934 - train acc 0.539 - valid loss 2.109 - valid acc 0.358
Epoch 7: train loss 1.917 - train acc 0.542 - valid loss 1.973 - valid acc 0.419
Epoch 8: train loss 1.831 - train acc 0.579 - valid loss 2.247 - valid acc 0.307
Epoch 9: train loss 1.802 - train acc 0.591 - valid loss 2.450 - valid acc 0.311
Epoch 10: train loss 1.776 - train acc 0.587 - valid loss 2.137 - valid acc 0.358
Epoch 11: train loss 1.728 - train acc 0.619 - valid loss 2.249 - valid acc 0.331
Epoch 12: train loss 1.694

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 0: train loss 3.407 - train acc 0.143 - valid loss 3.371 - valid acc 0.182
Epoch 1: train loss 3.177 - train acc 0.176 - valid loss 3.223 - valid acc 0.155
Epoch 2: train loss 3.143 - train acc 0.178 - valid loss 3.188 - valid acc 0.139
Epoch 3: train loss 3.377 - train acc 0.152 - valid loss 3.523 - valid acc 0.074
Epoch 4: train loss 3.494 - train acc 0.104 - valid loss 3.485 - valid acc 0.074
Epoch 5: train loss 3.477 - train acc 0.102 - valid loss 3.493 - valid acc 0.074
Epoch 6: train loss 3.476 - train acc 0.102 - valid loss 3.495 - valid acc 0.074
Epoch 7: train loss 3.476 - train acc 0.101 - valid loss 3.492 - valid acc 0.074
Epoch 8: train loss 3.477 - train acc 0.105 - valid loss 3.491 - valid acc 0.074
Epoch 9: train loss 3.660 - train acc 0.109 - valid loss 3.493 - valid acc 0.074
Epoch 10: train loss 3.476 - train acc 0.102 - valid loss 3.496 - valid acc 0.074
Epoch 11: train loss 3.477 - train acc 0.105 - valid loss 3.494 - valid acc 0.074
Epoch 12: train loss 3.475